# 样例介绍
* ResNet是最经典的视觉分类网络之一，在这个样例中我们选取了ResNet50，也是ResNet最常用的变体。

# 前期准备
* 基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可参考以下步骤。
* 首先我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，下载我们已经准备好了的Caffe模型文件，Caffe是一个开源深度学习框架。

* 为了能进一步优化模型推理性能，我们需要将其转换为om模型进行使用，以下为转换指令：  
    ```shell
    atc --model=resnet50.prototxt --weight=resnet50.caffemodel --framework=0 --output=resnet50 --input_format=NCHW --soc_version=Ascend310B1 --input_fp16_nodes=data --output_type=FP32 --out_nodes=prob:0
    ```
    * 其中转换参数的含义为：  
        * --model：输入模型路径
        * --weight：输入模型权重
        * --framework：原始网络模型框架类型，0表示Caffe
        * --output：输出模型路径
        * --soc_version：昇腾AI处理器型号
        * --input_format：输入Tensor的内存排列方式
        * --input_fp16_nodes：指定输入数据类型为FP16的输入节点名称
        * --output_type：指定网络输出数据类型或指定某个输出节点的输出类型
        * --out_nodes：指定输出节点

# 模型推理实现
* 导入代码依赖

In [ ]:
import json
import argparse
import os

import numpy as np
import acl
import cv2
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

from constant import ACL_MEM_MALLOC_HUGE_FIRST, \
    ACL_MEMCPY_HOST_TO_DEVICE, ACL_MEMCPY_DEVICE_TO_HOST, \
    ACL_SUCCESS, IMG_EXT, NPY_FLOAT32

* 主要推理代码

In [ ]:
buffer_method = {
    "in": acl.mdl.get_input_size_by_index,
    "out": acl.mdl.get_output_size_by_index
}


def check_ret(message, ret):
    """检查返回数据"""
    if ret != ACL_SUCCESS:
        raise Exception("{} failed ret={}"
                        .format(message, ret))


class Net(object):
    """推理网络定义"""
    def __init__(self, device_id, model_path, idx2label_list):
        self.device_id = device_id 
        self.model_path = model_path
        self.model_id = None
        self.context = None

        self.input_data = []
        self.output_data = []
        self.model_desc = None
        self.load_input_dataset = None
        self.load_output_dataset = None

        self.init_resource()
        self.idx2label_list = idx2label_list

    def release_resource(self):
        """释放ACL资源"""
        print("Releasing resources stage:")
        ret = acl.mdl.unload(self.model_id)
        check_ret("acl.mdl.unload", ret)
        if self.model_desc:
            acl.mdl.destroy_desc(self.model_desc)
            self.model_desc = None

        while self.input_data:
            item = self.input_data.pop()
            ret = acl.rt.free(item["buffer"])
            check_ret("acl.rt.free", ret)

        while self.output_data:
            item = self.output_data.pop()
            ret = acl.rt.free(item["buffer"])
            check_ret("acl.rt.free", ret)

        if self.context:
            ret = acl.rt.destroy_context(self.context)
            check_ret("acl.rt.destroy_context", ret)
            self.context = None

        ret = acl.rt.reset_device(self.device_id)
        check_ret("acl.rt.reset_device", ret)
        print('Resources released successfully.')

    def init_resource(self):
        """初始化ACL资源"""
        acl.init()
        print("init resource stage:")
        ret = acl.rt.set_device(self.device_id)
        check_ret("acl.rt.set_device", ret)

        self.context, ret = acl.rt.create_context(self.device_id)
        check_ret("acl.rt.create_context", ret)

        # load_model
        self.model_id, ret = acl.mdl.load_from_file(self.model_path)
        check_ret("acl.mdl.load_from_file", ret)
        print("model_id:{}".format(self.model_id))

        self.model_desc = acl.mdl.create_desc()
        self._get_model_info()
        print("init resource success")

    def _get_model_info(self,):
        """获取模型信息"""
        ret = acl.mdl.get_desc(self.model_desc, self.model_id)
        check_ret("acl.mdl.get_desc", ret)
        input_size = acl.mdl.get_num_inputs(self.model_desc)
        output_size = acl.mdl.get_num_outputs(self.model_desc)
        self._gen_data_buffer(input_size, des="in")
        self._gen_data_buffer(output_size, des="out")

    def _gen_data_buffer(self, size, des):
        """准备模型输入输出数据"""
        func = buffer_method[des]
        for i in range(size):
            # check temp_buffer dtype
            temp_buffer_size = func(self.model_desc, i)
            temp_buffer, ret = acl.rt.malloc(temp_buffer_size,
                                             ACL_MEM_MALLOC_HUGE_FIRST)
            check_ret("acl.rt.malloc", ret)

            if des == "in":
                self.input_data.append({"buffer": temp_buffer,
                                        "size": temp_buffer_size})
            elif des == "out":
                self.output_data.append({"buffer": temp_buffer,
                                         "size": temp_buffer_size})

    def _data_interaction(self, dataset, policy=ACL_MEMCPY_HOST_TO_DEVICE):
        """在host和device之间通过复制内存进行数据传输"""
        temp_data_buffer = self.input_data \
            if policy == ACL_MEMCPY_HOST_TO_DEVICE \
            else self.output_data
        if len(dataset) == 0 and policy == ACL_MEMCPY_DEVICE_TO_HOST:
            for item in self.output_data:
                temp, ret = acl.rt.malloc_host(item["size"])
                if ret != 0:
                    raise Exception("can't malloc_host ret={}".format(ret))
                dataset.append({"size": item["size"], "buffer": temp})

        for i, item in enumerate(temp_data_buffer):
            if policy == ACL_MEMCPY_HOST_TO_DEVICE:
                if "bytes_to_ptr" in dir(acl.util):
                    bytes_data = dataset[i].tobytes()
                    ptr = acl.util.bytes_to_ptr(bytes_data)
                else:
                    ptr = acl.util.numpy_to_ptr(dataset[i])
                ret = acl.rt.memcpy(item["buffer"],
                                    item["size"],
                                    ptr,
                                    item["size"],
                                    policy)
                check_ret("acl.rt.memcpy", ret)

            else:
                ptr = dataset[i]["buffer"]
                ret = acl.rt.memcpy(ptr,
                                    item["size"],
                                    item["buffer"],
                                    item["size"],
                                    policy)
                check_ret("acl.rt.memcpy", ret)

    def _gen_dataset(self, type_str="input"):
        """准备模型输入输出数据集"""
        dataset = acl.mdl.create_dataset()

        temp_dataset = None
        if type_str == "in":
            self.load_input_dataset = dataset
            temp_dataset = self.input_data
        else:
            self.load_output_dataset = dataset
            temp_dataset = self.output_data

        for item in temp_dataset:
            data = acl.create_data_buffer(item["buffer"], item["size"])
            _, ret = acl.mdl.add_dataset_buffer(dataset, data)

            if ret != ACL_SUCCESS:
                ret = acl.destroy_data_buffer(data)
                check_ret("acl.destroy_data_buffer", ret)

    def _data_from_host_to_device(self, images):
        """将数据从host传到device"""
        print("data interaction from host to device")
        # copy images to device
        self._data_interaction(images, ACL_MEMCPY_HOST_TO_DEVICE)
        # load input data into model
        self._gen_dataset("in")
        # load output data into model
        self._gen_dataset("out")
        print("data interaction from host to device success")

    def _data_from_device_to_host(self):
        """将数据从device传到host"""
        print("data interaction from device to host")
        res = []
        # copy device to host
        self._data_interaction(res, ACL_MEMCPY_DEVICE_TO_HOST)
        print("data interaction from device to host success")
        result = self.get_result(res)
        pred_dict = self._print_result(result)
        # free host memory
        for item in res:
            ptr = item['buffer']
            ret = acl.rt.free_host(ptr)
            check_ret('acl.rt.free_host', ret)
        return pred_dict

    def run(self, images):
        """执行数据传输和模型推理"""
        self._data_from_host_to_device(images)
        self.forward()
        pred_dict = self._data_from_device_to_host()
        return pred_dict

    def forward(self):
        """模型推理"""
        print('execute stage:')
        ret = acl.mdl.execute(self.model_id,
                              self.load_input_dataset,
                              self.load_output_dataset)
        check_ret("acl.mdl.execute", ret)
        self._destroy_databuffer()
        print('execute stage success')

    def _print_result(self, result):
        """打印推理结果"""
        vals = np.array(result).flatten()
        top_k = vals.argsort()[-1:-6:-1]
        print("======== top5 inference results: =============")
        
        pred_dict = {}
        for j in top_k:
            print(f'{self.idx2label_list[j]}: {vals[j]}')
            pred_dict[self.idx2label_list[j]] = vals[j]
        return pred_dict

    def _destroy_databuffer(self):
        """释放模型输入输出数据"""
        for dataset in [self.load_input_dataset, self.load_output_dataset]:
            if not dataset:
                continue
            number = acl.mdl.get_dataset_num_buffers(dataset)
            for i in range(number):
                data_buf = acl.mdl.get_dataset_buffer(dataset, i)
                if data_buf:
                    ret = acl.destroy_data_buffer(data_buf)
                    check_ret("acl.destroy_data_buffer", ret)
            ret = acl.mdl.destroy_dataset(dataset)
            check_ret("acl.mdl.destroy_dataset", ret)

    def get_result(self, output_data):
        """获取模型输出结果"""
        result = []
        dims, ret = acl.mdl.get_cur_output_dims(self.model_desc, 0)
        check_ret("acl.mdl.get_cur_output_dims", ret)
        out_dim = dims['dims']
        for temp in output_data:
            ptr = temp["buffer"]
            # 转化为float32类型的数据
            if "ptr_to_bytes" in dir(acl.util):
                bytes_data = acl.util.ptr_to_bytes(ptr, temp["size"])
                data = np.frombuffer(bytes_data, dtype=np.float32).reshape(tuple(out_dim))
            else:
                data = acl.util.ptr_to_numpy(ptr, tuple(out_dim), NPY_FLOAT32)
            result.append(data)
        return result


def preprocess_img(input_path):
    """图片预处理"""
    input_path = os.path.abspath(input_path)
    with Image.open(input_path) as image_file:
        image_file = image_file.resize((256, 256))
        img = np.array(image_file)
    height = img.shape[0]
    width = img.shape[1]
    # 对图片进行切分，取中间区域
    h_off = (height - 224) // 2
    w_off = (width - 224) // 2
    crop_img = img[h_off:height - h_off, w_off:width - w_off, :]
    # rgb to bgr，改变通道顺序
    img = crop_img[:, :, ::-1]
    shape = img.shape
    img = img.astype("float16")
    # Caffe网络的数据标准化
    img[:, :, 0] -= 104
    img[:, :, 1] -= 117
    img[:, :, 2] -= 123
    img = img.reshape([1] + list(shape))
    img = img.transpose([0, 3, 1, 2])
    result = np.frombuffer(img.tobytes(), np.float16)
    return result


def load_imagenet_labels(path):
    with open(path) as f:
        label_list = json.load(f)
    return label_list


def display_image(path, pred_dict):
    setFont = ImageFont.truetype('font.ttf', 20)
    fillColor = "#fff"
    im = Image.open(path)
    im = im.resize((800, 500))
    draw = ImageDraw.Draw(im)
    
    start_y = 20
    for label, pred in pred_dict.items():
        draw.text(xy = (20, start_y), text=f'{label}: {pred:.2f}', font=setFont, fill=fillColor)
        start_y += 30

    plt.axis('off')
    plt.xticks([])
    plt.yticks([])
    plt.imshow(im)
    
    
def main():
    device = 0
    model_path = 'model/resnet50.om'
    images_path = './data'
    label_path = './imagenet-simple-labels.json'
    
    idx2label_list = load_imagenet_labels(label_path)
    net = Net(device, model_path, idx2label_list)
    images_list = [os.path.join(images_path, img)
                   for img in os.listdir(images_path)
                   if os.path.splitext(img)[1] in IMG_EXT]
    
    for image in images_list:
        print("images:{}".format(image))
        img = preprocess_img(image)
        pred_dict = net.run([img])
        display_image(image, pred_dict)
    
    print("*****run finish******")
    net.release_resource()

# 样例运行

* 运行以下代码后，我们可以看到在推理结果中，展示了5个类别的推理概率，其中概率最高的类别是Standard Poodle，概率约为0.94，该类别正是对应样例图片中的小狗了。

In [ ]:
main()

# 样例总结与扩展
以上就是这个样例的全部内容了，ResNet50是一个基础的图像分类网络，相信大家通过这个样例的学习得到了更深入的掌握，同时也熟悉了ACL的推理流程。